## Подготовка

In [1]:
import numpy as np
import pandas as pd

import json
import requests
from bs4 import BeautifulSoup
import time

In [2]:
names = ['id', 'header', 'date',
         'newspaper', 'city', 'md_index', 'url']

data = pd.read_excel('urls-for-parsing.xlsx',
                     names = names)
data.head(3)

,id,header,date,newspaper,city,md_index,url
703,704,Женщины для утех по-прежнему нужны: Япония и К...,2016-01-03,ИА Regnum,Москва,2.312,http://regnum.ru/news/2048238.html
700,701,В Норвегии мигрантов научат не насиловать женщин,2016-01-04,Известия (iz.ru),Москва,1.113,http://izvestia.ru/news/600956
693,694,Полиция Кельна опровергла сообщения о нападени...,2016-01-05,Российская газета (rg.ru),Москва,1.266,http://www.rg.ru/2016/01/05/keln-site.html


In [3]:
def parser(links, scraper):

    headlines = []
    descriptions = []
    article_bodies = []
    urls = []

    for link in links:
      try:
        h, d, a = scraper(link)
        headlines.append(h)
        descriptions.append(d)
        article_bodies.append(a)
        urls.append(link)

      except TypeError:
        pass
          
    return(headlines, descriptions, article_bodies, urls)

In [4]:
def save_texts(name, texts):  
    out_path = 'C:/Users/KseniaLapshina/thesis-data-collection/texts-files-api/' + name + '.xlsx'
    writer = pd.ExcelWriter(out_path , engine='xlsxwriter')
    texts.to_excel(writer)
    writer.close()

In [5]:
cols = ['headline', 'description', 'article_body', 'url']
texts_all = pd.DataFrame(columns = cols)
texts_all

,headline,description,article_body,url


In [43]:
# Временный апи ключ на аккаунте, лимит запросов - 1К на аккаунт
apikey = 'cc34d1d7dd07bcc509c9fe45014fe6c164d47005'
apikey = '8aced2c0cc35f3e261c331f90555b0ecb563ff87'

## Известия (iz.ru)

In [7]:
name = 'Известия (iz.ru)'

In [8]:
def iz_scraper(link):

    params = {'url': link, 'apikey': apikey}
    
    try:
        try:
          
          r = requests.get('https://api.zenrows.com/v1/', params=params)
        except requests.exceptions.RequestException as ex:
          print(ex)
    
        soup = BeautifulSoup(r.content, "html.parser")
    
        headline = soup.find('meta', property="og:title").get("content")
        description = soup.find('meta', property="og:description").get("content")
        article_body = ''
    
        blocks = soup.find_all('p')
        for block in blocks:
          article_body = article_body + block.get_text()
    
        return(headline, description, article_body)
    
    except (UnboundLocalError, KeyError, AttributeError, UnicodeDecodeError) as error:
      print(error)
      pass

In [9]:
links_iz = data[data.newspaper == name].url
links_iz.shape

(422,)

In [10]:
%%time
headlines, descriptions, article_bodies, urls = parser(links_iz, iz_scraper)

CPU times: total: 5min 56s
Wall time: 23min 31s


In [11]:
d = {'headline': headlines, 'description': descriptions, 'article_body': article_bodies, 'url' : urls}
texts_iz = pd.DataFrame(data=d)
texts_iz.shape

(422, 4)

In [12]:
save_texts(name, texts_iz)
texts_all = pd.concat([texts_all, texts_iz])

## NEWS.ru

In [13]:
name = 'NEWS.ru'

In [14]:
def ne_scraper(link):

    params = {'url': link, 'apikey': apikey}
    
    try:
        try:
          r = requests.get('https://api.zenrows.com/v1/', params=params)
        except requests.exceptions.RequestException as ex:
          print(ex)
    
        soup = BeautifulSoup(r.content, "html.parser")
    
        headline = soup.find('meta', property="og:title").get("content")
        description = soup.find('meta', property="og:description").get("content")
        article_body = soup.find('div', {'itemprop' : 'articleBody'}).get_text()
    
        return(headline, description, article_body)
    
    except (UnboundLocalError, KeyError, AttributeError, UnicodeDecodeError) as error:
      print(error)
      pass

In [15]:
links_ne = data[data.newspaper == name].url
links_ne.shape

(386,)

In [16]:
%%time
headlines, descriptions, article_bodies, urls = parser(links_ne, ne_scraper)

'NoneType' object has no attribute 'get'
CPU times: total: 5min 6s
Wall time: 16min 16s


In [17]:
d = {'headline': headlines, 'description': descriptions, 'article_body': article_bodies, 'url' : urls}
texts_ne = pd.DataFrame(data=d)
texts_ne.shape

(385, 4)

In [18]:
save_texts(name, texts_ne)
texts_all = pd.concat([texts_all, texts_ne])

## Фонтанка (fontanka.ru)

In [44]:
name = 'Фонтанка (fontanka.ru)'

In [45]:
def fn_scraper(link):

    params = {'url': link, 'apikey': apikey}
    
    try:
        try:
          r = requests.get('https://api.zenrows.com/v1/', params=params)
        except requests.exceptions.RequestException as ex:
          print(ex)
    
        soup = BeautifulSoup(r.content, "html.parser")
    
        headline = soup.find('meta', property="og:title").get("content")
        description = soup.find('meta', property="og:description").get("content")
    
        article_body = ''
        blocks = soup.find_all('p')
        for block in blocks:
          article_body = article_body + block.get_text()
    
        return(headline, description, article_body)
    
    except (UnboundLocalError, KeyError, AttributeError, UnicodeDecodeError) as error:
      print(error)
      pass

In [46]:
links_fn = data[data.newspaper == name].url
links_fn.shape

(341,)

In [47]:
%%time
headlines, descriptions, article_bodies, urls = parser(links_fn, fn_scraper)

'NoneType' object has no attribute 'get'
'NoneType' object has no attribute 'get'
'NoneType' object has no attribute 'get'
'NoneType' object has no attribute 'get'
'NoneType' object has no attribute 'get'
'NoneType' object has no attribute 'get'
'NoneType' object has no attribute 'get'
'NoneType' object has no attribute 'get'
'NoneType' object has no attribute 'get'
CPU times: total: 4min 55s
Wall time: 21min 11s


In [48]:
d = {'headline': headlines, 'description': descriptions, 'article_body': article_bodies, 'url' : urls}
texts_fn = pd.DataFrame(data=d)
texts_fn.shape

(332, 4)

In [49]:
save_texts(name, texts_fn)
texts_all = pd.concat([texts_all, texts_fn])

## Чемпионат.com (championat.com)

In [50]:
name = 'Чемпионат.com (championat.com)'

In [51]:
def ch_scraper(link):

    params = {'url': link, 'apikey': apikey}
    
    try:
        try:
          r = requests.get('https://api.zenrows.com/v1/', params=params)
        except requests.exceptions.RequestException as ex:
          print(ex)
    
        soup = BeautifulSoup(r.content, "html.parser")
    
        headline = soup.find('meta', property="og:title").get("content")
        description = soup.find('meta', property="og:description").get("content")
        article_body = soup.find('div', {'id': 'articleBody'}).get_text()
    
        return(headline, description, article_body)
    
    except (UnboundLocalError, KeyError, AttributeError, UnicodeDecodeError) as error:
      print(error)
      pass

In [52]:
links_ch = data[data.newspaper == name].url
links_ch.shape

(242,)

In [53]:
%%time
headlines, descriptions, article_bodies, urls = parser(links_ch, ch_scraper)

'NoneType' object has no attribute 'get'
'NoneType' object has no attribute 'get'
'NoneType' object has no attribute 'get'
'NoneType' object has no attribute 'get'
'NoneType' object has no attribute 'get'
CPU times: total: 3min 30s
Wall time: 16min 13s


In [54]:
d = {'headline': headlines, 'description': descriptions, 'article_body': article_bodies, 'url' : urls}
texts_ch = pd.DataFrame(data=d)
texts_ch.shape

(237, 4)

In [55]:
save_texts(name, texts_ch)
texts_all = pd.concat([texts_all, texts_ch])

## V1.ru

In [25]:
name = 'V1.ru'

In [26]:
def v1_scraper(link):

    params = {'url': link, 'apikey': apikey}
    
    try:
        try:
          r = requests.get('https://api.zenrows.com/v1/', params=params)
        except requests.exceptions.RequestException as ex:
          print(ex)
    
        soup = BeautifulSoup(r.content, "html.parser")
    
        headline = soup.find('title').get_text()
        description = soup.find('meta', {'name' : 'description'}).get("content")
        article_body = ''
    
        blocks = soup.find_all('p')
        for block in blocks:
          article_body = article_body + block.get_text()
    
        return(headline, description, article_body)
    
    except (UnboundLocalError, KeyError, AttributeError, UnicodeDecodeError) as error:
      print(error)
      pass

In [27]:
links_v1 = data[data.newspaper == name].url
links_v1.shape

(90,)

In [28]:
%%time
headlines, descriptions, article_bodies, urls = parser(links_v1, v1_scraper)

CPU times: total: 1min 14s
Wall time: 6min


In [29]:
d = {'headline': headlines, 'description': descriptions, 'article_body': article_bodies, 'url' : urls}
texts_v1 = pd.DataFrame(data=d)
texts_v1.shape

(90, 4)

In [30]:
save_texts(name, texts_v1)
texts_all = pd.concat([texts_all, texts_v1])

## Екатеринбург Он-лайн (e1.ru)

In [31]:
name = 'Екатеринбург Он-лайн (e1.ru)'

In [32]:
def e1_scraper(link):

    params = {'url': link, 'apikey': apikey}
    
    try:
        try:
          r = requests.get('https://api.zenrows.com/v1/', params=params)
        except requests.exceptions.RequestException as ex:
          print(ex)
    
        soup = BeautifulSoup(r.content, "html.parser")
    
        headline = soup.find('title').get_text()
        description = soup.find('meta', {'name' : 'description'}).get("content")
        article_body = ''
    
        blocks = soup.find_all('p')
        for block in blocks:
          article_body = article_body + block.get_text()
    
        return(headline, description, article_body)
    
    except (UnboundLocalError, KeyError, AttributeError, UnicodeDecodeError) as error:
      print(error)
      pass

In [33]:
links_e1 = data[data.newspaper == name].url
links_e1.shape

(91,)

In [34]:
%%time
headlines, descriptions, article_bodies, urls = parser(links_e1, e1_scraper)

'NoneType' object has no attribute 'get_text'
'NoneType' object has no attribute 'get_text'
'NoneType' object has no attribute 'get_text'
'NoneType' object has no attribute 'get_text'
CPU times: total: 1min 21s
Wall time: 5min 4s


In [35]:
d = {'headline': headlines, 'description': descriptions, 'article_body': article_bodies, 'url' : urls}
texts_e1 = pd.DataFrame(data=d)
texts_e1.shape

(87, 4)

In [36]:
save_texts(name, texts_e1)
texts_all = pd.concat([texts_all, texts_e1])

## 59.ru

In [62]:
name = '59.ru'

In [63]:
def scraper_59(link):

    params = {'url': link, 'apikey': apikey}
    
    try:
        try:
          r = requests.get('https://api.zenrows.com/v1/', params=params)
        except requests.exceptions.RequestException as ex:
          print(ex)
    
        soup = BeautifulSoup(r.content, "html.parser")
    
        headline = soup.find('title').get_text()
        description = soup.find('meta', {'name' : 'description'}).get("content")
        article_body = soup.find('div', {'itemprop': 'articleBody'}).get_text()
    
        return(headline, description, article_body)
    
    except (UnboundLocalError, KeyError, AttributeError, UnicodeDecodeError) as error:
      print(error)
      pass

In [64]:
links_59 = data[data.newspaper == name].url
links_59.shape

(78,)

In [65]:
%%time
headlines, descriptions, article_bodies, urls = parser(links_59, scraper_59)

CPU times: total: 1min 5s
Wall time: 6min


In [66]:
d = {'headline': headlines, 'description': descriptions, 'article_body': article_bodies, 'url' : urls}
texts_59 = pd.DataFrame(data=d)
texts_59.shape

(78, 4)

In [67]:
save_texts(name, texts_59)
texts_all = pd.concat([texts_all, texts_59])

## 72.ru

In [69]:
name = '72.ru'

In [70]:
def scraper_72(link):

    params = {'url': link, 'apikey': apikey}
    
    try:
        try:
          r = requests.get('https://api.zenrows.com/v1/', params=params)
        except requests.exceptions.RequestException as ex:
          print(ex)
    
        soup = BeautifulSoup(r.content, "html.parser")
    
        headline = soup.find('title').get_text()
        description = soup.find('meta', {'name' : 'description'}).get("content")
        article_body = ''
    
        blocks= soup.find_all('p')
        for block in blocks:
          article_body = article_body + block.get_text()
    
        return(headline, description, article_body)
    
    except (UnboundLocalError, KeyError, AttributeError, UnicodeDecodeError) as error:
      print(error)
      pass

In [71]:
links_72 = data[data.newspaper == name].url
links_72.shape

(61,)

In [72]:
%%time
headlines, descriptions, article_bodies, urls = parser(links_72, scraper_72)

CPU times: total: 52.1 s
Wall time: 3min 56s


In [73]:
d = {'headline': headlines, 'description': descriptions, 'article_body': article_bodies, 'url' : urls}
texts_72 = pd.DataFrame(data=d)
texts_72.shape

(61, 4)

In [74]:
save_texts(name, texts_72)
texts_all = pd.concat([texts_all, texts_72])

## Финальный датасет

In [75]:
texts_all = texts_all.drop_duplicates()
texts_all.head(3)

,headline,description,article_body,url
0,В Норвегии мигрантов научат не насиловать женщин,Наплыв беженцев из Сирии и других стран Ближне...,\n ...,http://izvestia.ru/news/600956
1,Серийному маньяку из Севастополя предъявлено о...,Павла Бондаренко обвиняют в убийстве четырех ж...,\n ...,http://izvestia.ru/news/603261
2,На карнавале в Кельне сексуальным домогательст...,"Из 22 случаев наиболее серьезными стали два, к...",\n ...,http://izvestia.ru/news/603491


In [76]:
texts_all.to_excel('texts-parsed-api.xlsx')